In [1]:
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler
from sklearn import metrics
from sklearn.model_selection import train_test_split, GridSearchCV
from statsmodels.api import Logit
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import confusion_matrix

import os
os.environ["PATH"] += os.pathsep + "C:/Program Files/Graphviz2.44.1/bin/"
from sklearn.tree import export_graphviz
import graphviz

In [2]:
# 한글 폰트 적용
from matplotlib import font_manager, rc
# font_fname = 'C:/Users/tjrdl/Desktop/Posco_AI_Bigdata_11th/Bigdata/NanumGothic.ttf'
font_fname = 'C:/Users/tjrdl/AppData/Local/Microsoft/Windows/Fonts/NanumBarunGothic.otf'
font_family = font_manager.FontProperties(fname=font_fname).get_name()
matplotlib.rc('font', family=font_fname)

%matplotlib inline
plt.rcParams["font.family"] = font_family

In [3]:
df_order = pd.read_csv("log_order_sheet_000.csv", engine="python")
df_member = pd.read_csv("master_member_sheet_000.csv", engine="python")
df_product = pd.read_csv("master_product_sheet_000.csv", engine="python")

In [28]:
df_order.head()

,결제수단,공급원가,교환구분,네이버 포인트,배송번호,배송비 정보,배송시작일,배송완료일,사용한 적립금액,상품구매금액,...,수령인 주소,주문 시 회원등급,주문경로,주문번호,주문일시,주문자ID,카드사,쿠폰 할인금액,판매가,품목번호
0,무통장입금,44550.0,교환안함,0,D-20181227-0000648-00,무료,2019-01-07 오전 8:56,2019-01-11 오전 5:32,0,71450,...,경상남도 거제시 능포로 168-1 (능포동),전화주문 고객,PC쇼핑몰,20181227-0000648,2018-12-27 오후 2:54,pp683,NaN,0,71450,42602
1,무통장입금,44000.0,교환안함,0,D-20181229-0000119-00,무료,2019-01-03 오전 9:30,2019-01-07 오전 5:32,0,141240,...,서울특별시 동대문구 한천로36길 16 (장안동),일반,모바일웹,20181229-0000119,2018-12-29 오후 1:22,asd0908,NaN,0,70620,42930
2,무통장입금,6820.0,교환안함,0,D-20181230-0000100-00,무료,2019-01-03 오전 9:30,2019-01-07 오전 5:32,0,13910,...,세종특별자치시 호려울로 29 (보람동),NaN,모바일웹,20181230-0000100,2018-12-30 오후 3:15,NaN,NaN,0,13910,42998
3,무통장입금,49500.0,교환안함,0,D-20181231-0000087-00,무료,2019-01-03 오전 9:30,2019-01-07 오전 5:32,0,81600,...,부산광역시 부산진구 가야대로 482 (개금동),일반,모바일웹,20181231-0000087,2018-12-31 오전 10:30,23533593@n,NaN,0,81600,43063
4,무통장입금,61600.0,교환안함,0,D-20181231-0000108-00,무료,2019-01-07 오전 8:56,2019-01-11 오전 5:32,0,94710,...,부산광역시 해운대구 선수촌로 104-21 (반여동),사업자,모바일웹,20181231-0000108,2018-12-31 오전 10:43,romandoll,NaN,0,94710,43066


In [81]:
df_order.isnull().sum()

결제수단              0
공급원가             88
교환구분              0
네이버 포인트           0
배송번호              0
배송비 정보            0
배송시작일             0
배송완료일             0
사용한 적립금액          0
상품구매금액            0
상품명              88
상품번호             88
상품별 추가할인금액        0
수량                0
수령인 주소            8
주문 시 회원등급      3105
주문경로              0
주문번호              0
주문일시              0
주문자ID          3105
카드사           30138
쿠폰 할인금액           0
판매가               0
품목번호              0
dtype: int64

## 결측,이상치 정제

1. 공급원가,상품명, 상품번호 결측치는 공백으로 확인 ==> 88개 row 제거
2. 배송지 주소 결측치는 공백으로 확인 ==> 8개 row 제거
3. 사은품 row는 모두 제거
4. 주문 시 회원등급 결측치는 비회원으로 대체

카드 이용한 결제 중 카드사 결측치는 없음.(결측치는 모두 카드결제 이외 결제 방법임)

## 제거 변수
### 사유
네이버 포인트 사용여부<br>
사용한 적립금액<br>
상품별 추가할인금액<br>
배송번호<br>
-----> 불필요

교환구분 ==> 전부 '교환안함'<br>
배송비 정보 ==> 전부 '무료'<br>
각종 고유 번호들

## 파생 변수
+ 주문일시, 배송시작일, 배송완료일 세가지로 "리드 타임" 생성
+ 수령인 주소는 "지역별 평균 배송소요일" 생성

In [102]:
# 상품명에 사은품은 모두 제거
mask = df_order['상품명'].isin(['사은품']) 
df_order = df_order[~mask]

In [103]:
df_order.dropna(subset=['상품명'], inplace=True)
df_order.dropna(subset=['수령인 주소'], inplace=True)
df_order["주문 시 회원등급"].fillna("비회원", inplace=True)

In [104]:
df_order.isnull().sum()

결제수단              0
공급원가              0
교환구분              0
네이버 포인트           0
배송번호              0
배송비 정보            0
배송시작일             0
배송완료일             0
사용한 적립금액          0
상품구매금액            0
상품명               0
상품번호              0
상품별 추가할인금액        0
수량                0
수령인 주소            0
주문 시 회원등급         0
주문경로              0
주문번호              0
주문일시              0
주문자ID          3063
카드사           30100
쿠폰 할인금액           0
판매가               0
품목번호              0
dtype: int64

In [106]:
df_order_scaled = df_order.drop(['교환구분','네이버 포인트','배송번호','배송비 정보','사용한 적립금액','상품별 추가할인금액',
                                 '주문번호','주문자ID','카드사','쿠폰 할인금액','품목번호'], axis=1)

In [107]:
df_order_scaled.dtypes

결제수단          object
공급원가         float64
배송시작일         object
배송완료일         object
상품구매금액         int64
상품명           object
상품번호         float64
수량             int64
수령인 주소        object
주문 시 회원등급     object
주문경로          object
주문일시          object
판매가            int64
dtype: object

In [108]:
df_order_scaled = df_order_scaled.astype({'상품번호':int})

In [109]:
df_order_scaled.head()

,결제수단,공급원가,배송시작일,배송완료일,상품구매금액,상품명,상품번호,수량,수령인 주소,주문 시 회원등급,주문경로,주문일시,판매가
0,무통장입금,44550.0,2019-01-07 오전 8:56,2019-01-11 오전 5:32,71450,195파이 탕용기 소 300개 1set(2박스),1077,1,경상남도 거제시 능포로 168-1 (능포동),전화주문 고객,PC쇼핑몰,2018-12-27 오후 2:54,71450
1,무통장입금,44000.0,2019-01-03 오전 9:30,2019-01-07 오전 5:32,141240,"95파이 다용도컵 세트 4호 소 200ml 백색 PS 1,000개 1set(2박스)",42,2,서울특별시 동대문구 한천로36길 16 (장안동),일반,모바일웹,2018-12-29 오후 1:22,70620
2,무통장입금,6820.0,2019-01-03 오전 9:30,2019-01-07 오전 5:32,13910,비닐봉투 배달중 중 백색 200개,1271,1,세종특별자치시 호려울로 29 (보람동),비회원,모바일웹,2018-12-30 오후 3:15,13910
3,무통장입금,49500.0,2019-01-03 오전 9:30,2019-01-07 오전 5:32,81600,195파이 미니탕용기 투톤 300개 1set(2박스),1085,1,부산광역시 부산진구 가야대로 482 (개금동),일반,모바일웹,2018-12-31 오전 10:30,81600
4,무통장입금,61600.0,2019-01-07 오전 8:56,2019-01-11 오전 5:32,94710,250파이 원형 접시 미니 블랙 400개 1박스,1106,1,부산광역시 해운대구 선수촌로 104-21 (반여동),사업자,모바일웹,2018-12-31 오전 10:43,94710


# product 

### 제조사 결측치 패커스 자체제작으로 처리

In [11]:
df_product.isnull().sum()

#            0
공급사          0
과세/면세        0
담기           0
매입가          0
상품명(기본)      0
상품코드         0
재입고알림        0
제조사        486
카테고리         0
판매가          0
품절여부         0
후기           0
dtype: int64

In [13]:
df_product["제조사"].fillna("자체제작", inplace=True)

In [14]:
df_product.isnull().sum()

#          0
공급사        0
과세/면세      0
담기         0
매입가        0
상품명(기본)    0
상품코드       0
재입고알림      0
제조사        0
카테고리       0
판매가        0
품절여부       0
후기         0
dtype: int64

# member 


In [15]:
df_member.isnull().sum()

ID              0
나이              0
사용가능 적립금        0
성별              0
주소(동/읍/면)     634
최종접속일          52
최종주문일        1422
유입경로            0
유입기기            0
회원 가입일          0
사업자구분           0
회원구분            0
dtype: int64

최종주문일 결측치는 아이디는 있-지만 한 번도 주문을 한적이 없는 유저들<br>
==> 프로모션 보내주는 아이디어

In [ ]:
df_member['최종주문일'].fillna(df['최종접속일'], inplace=True)

In [9]:
df_member['나이'].mean()

40.921977186311786